In [ ]:
from __future__ import annotations
__authors__: list[str] = ['Rahul_Sawhney', 'nikhil_kumar_pradhan']

# Python Imports
import typing
from typing import Any, NewType, Generator, Optional, Union, ClassVar, Container
import os, warnings, sys
warnings.filterwarnings(action= 'ignore')


# Data Analysis Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.transforms import functional as tFF


# DL Imports 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


# torch typing scripts
_path =  NewType('_path', Any)
_transform = NewType('_transform', Any)
_img_path = NewType('_img', Any)
_criterion = NewType('_criterion', Any)
_optimizer = NewType('_optimizer', Any)
_loss = NewType('_loss', Any)
_layer = NewType('_layer', Any)
_activation = NewType('_activation', Any)
_text = NewType('_text', Any)
_plot = NewType('_plot', Any)
_loader = NewType('_loader', Any)
_recurse = NewType('_recurse', Any)


from tqdm import tqdm

## _class DataSet_

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@: Class DataSet
class BrainDataset(torch.utils.data.Dataset):
    def __init__(self, path: _path, sub_path: str, 
                                    categories: list[str], 
                                    img_resolution: Optional[int] = 64,
                                    transform: Optional[Container[Module[_transform]]] = None) -> None:
        self.path = path
        self.sub_path = sub_path
        self.categories = categories
        self.img_resolution = img_resolution
        if transform:
            self.transform = transform
        self.dataset: pd.DataFrame[_img_path, int] = self.get_data(path, sub_path, self.categories)
    


    @classmethod
    def get_data(cls, path: _path, sub_path: str, categories: list[str]) -> pd.DataFrame[_img_path, int]:
        # #@: local path
        # glioma_tumor: _path = path + sub_path + '\\' + categories[0] + '\\'
        # meningioma_tumor: _path = path + sub_path + '\\' + categories[1] + '\\'
        # no_tumor: _path = path + sub_path + '\\' + categories[2] + '\\'
        # pituitary_tumor: _path = path + sub_path + '\\' + categories[3] + '\\'


        ##@: colab path
        glioma_tumor: _path =  '/content/drive/MyDrive/BRAIN_MRI/Testing/glioma_tumor'
        meningioma_tumor: _path = '/content/drive/MyDrive/BRAIN_MRI/Testing/meningioma_tumor'
        no_tumor: _path = '/content/drive/MyDrive/BRAIN_MRI/Testing/no_tumor'
        pituitary_tumor: _path = '/content/drive/MyDrive/BRAIN_MRI/Testing/pituitary_tumor'
    


        glioma_tumor_img_path: list[_img_path] = [
            os.path.abspath(os.path.join(glioma_tumor, p)) for p in os.listdir(glioma_tumor)
        ] 
        meningioma_tumor_img_path: list[_img_path] = [
            os.path.abspath(os.path.join(meningioma_tumor, p)) for p in os.listdir(meningioma_tumor)
        ]
        no_tumor_img_path: list[_img_path] = [
            os.path.abspath(os.path.join(no_tumor, p)) for p in os.listdir(no_tumor)
        ]
        pituitary_tumor_img_path: list[_img_path] = [
            os.path.abspath(os.path.join(pituitary_tumor, p)) for p in os.listdir(pituitary_tumor)
        ]


        glioma_tumor_label: list[int] = [0 for _ in range(len(glioma_tumor_img_path))]
        meningioma_tumor_label: list[int] = [1 for _ in range(len(meningioma_tumor_img_path))]
        no_tumor_label: list[int] = [2 for _ in range(len(no_tumor_img_path))]
        pituitary_tumor_label: list[int] = [3 for _ in range(len(pituitary_tumor_img_path))]


        # pd.Dataframe[_img_path, int]  
        all_img_path: list[_img_path] = glioma_tumor_img_path + meningioma_tumor_img_path + no_tumor_img_path + pituitary_tumor_img_path
        all_label: list[int] = glioma_tumor_label + meningioma_tumor_label + no_tumor_label + pituitary_tumor_label

        dataframe: pd.DataFrame[_img_path, int] = pd.DataFrame.from_dict({'path' : all_img_path, 'label': all_label})
        dataframe: pd.DataFrame[_img_path, int] = dataframe.sample(frac= 1)
        return dataframe



    def __repr__(self) -> str(dict[str, str]):
        return str({
            item: value for item, value in zip(['Module', 'Name', 'ObjectID'],
                                               [self.__module__, type(self).__name__, hex(id(self))])
        })

    
    __str__ = __repr__


    def __len__(self) -> int:
        return len(self.dataset)


    def __getitem__(self, index: int) -> tuple[_img, int]:
        img_size: tuple[int, ...] = (self.img_resolution, self.img_resolution)
        image: _img = Image.open(self.dataset.iloc[index].path).convert('LA').resize(img_size)
        label: int = self.dataset.iloc[index].label 
        if self.transform:
            image: torch.Tensor = self.transform(image)
        
        return image, label  

## _class DataAnalysis_ 

In [ ]:
class BrainAnalysis:
    labels_map: ClassVar[dict[int, str]] = {
        0: 'Glioma Tumor',
        1: 'Meningioma Tumor',
        2: 'No Tumor',
        3: 'Pituitary Tumor'
    }

    def __repr__(self) -> str(dict[str, str]):
        return str({
            item: value for item, value in zip(['Module', 'Name', 'ObjectID'],
                                               [self.__module__, type(self).__name__, hex(id(self))])
        })


    __str__ = __repr__


    
    @classmethod
    def batch_img_display(cls, training_data: object) -> _plot:
        figure: plt.figure = plt.figure(figsize= (8, 8))
        cols, rows = 3, 3
        for i in range(1, cols * rows + 1):
            sample_index: int = torch.randint(len(training_data), size= (1,)).item()
            img, label = training_data[sample_index]
            figure.add_subplot(rows, cols, i)
            plt.title(cls.labels_map[label])
            plt.axis('off')
            plt.imshow(np.asarray(tFF.to_pil_image(img).convert('RGB')), cmap= 'gray')
        plt.show() 



    @classmethod
    def data_loader_img_display(cls, training_loader: _loader) -> _plot:
        train_feature, train_label = iter(training_loader).__next__()
        image: _img = np.asarray(tFF.to_pil_image(train_feature[0]).convert('RGB'))
        label: str = train_label[0]
        plt.title(cls.labels_map[int(label.numpy())])
        plt.imshow(image, cmap= 'gray')
        plt.show()


# _class Data Preprocess_

In [ ]:
class BrainPreprocess:
    def __repr__(self) -> str(dict[str, str]):
        return str({
            item: value for item, value in zip(['Module', 'Name', 'ObjectID'],
                                               [self.__module__, type(self).__name__, hex(id(self))])
        })

    __str__ = __repr__


    @classmethod
    def train_transform_container(cls) -> Container[Module[_transform]]:
        container: Container[Module[_transform]] = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomCrop(256, padding= 4, padding_mode= 'reflect'),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()
        ])
        return container


    @classmethod
    def test_transform_container(cls) -> Container[Module[_transform]]:
        container: Container[Module[_transform]] = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor()
        ])
        return container

## _Class GPU Acceleration_

In [ ]:
class GPU_Accerlaration:
    def __init__(self, train_loader: _loader, device: str) -> None:
        self.train_loader = train_loader
        self.device = device

    
    def __repr__(self) -> str(dict[str, str]):
        return str({
            item: value for item, value in zip(['Module', 'Name', 'ObjectID'],
                                               [self.__module__, type(self).__name__, hex(id(self))])
        })

    
    __str__ = __repr__


    @classmethod
    def to_device(cls, data: torch.Tensor, device: str) -> _recurse | [base, str]:
        if isinstance(data, (list, tuple)):
            return [cls.to_device(x, device) for x in data]
        return data.to(device, non_blocking= True)


    def __len__(self) -> int:
        return len(self.train_loader)


    def __iter__(self) -> Generator[_loader, None, None]:
        for batch in self.train_loader:
            yield self.to_device(batch, self.device)
    

    @classmethod
    def is_working_gpu(cls) -> bool:
        return True if cls.get_default_device() == 'cuda' else False


## _class CNN Paramters_

In [ ]:
class CNNHyperParams:
    #@: Hyper_params
    epochs: int = 5
    optimizer: _optimizor = torch.optim.Adam                
    #learning_rate: float = 0.01
    criterion: _criterion = nn.CrossEntropyLoss()
    weight_decay: float = 0.01
    momentum: float = 0.9


    #@: Layers
    convolutional_1: _layer = nn.Conv2d(in_channels= 2, out_channels= 32, 
                                        kernel_size= (3, 3), stride= 1, 
                                        padding= 0, dilation= 1, 
                                        groups= 1, bias= True, 
                                        padding_mode= 'zeros')
    
    convolutional_2: _layer = nn.Conv2d(in_channels= 32, out_channels= 64,
                                        kernel_size= (3, 3), stride= 1, 
                                        padding= 0, dilation= 1, 
                                        groups= 1, bias= True, 
                                        padding_mode= 'zeros')                                          
    

    convolutional_3: _layer = nn.Conv2d(in_channels= 64, out_channels= 128,  
                                        kernel_size= (3, 3), stride= 1, 
                                        padding= 0, dilation= 1, 
                                        groups= 1, bias= True, 
                                        padding_mode= 'zeros')
    
    convolutional_4: _layer = nn.Conv2d(in_channels= 128, out_channels= 256, 
                                        kernel_size= (3, 3), stride= 1, 
                                        padding= 0, dilation= 1, 
                                        groups= 1, bias= True, 
                                        padding_mode= 'zeros')

    
    pooling_1: _layer = nn.MaxPool2d(kernel_size= (2, 2), stride= 1,        
                                     padding= 0, dilation= 1,
                                     return_indices= False)
                                    
    
    pooling_2: _layer = nn.MaxPool2d(kernel_size= (2, 2), stride= 1,
                                     padding= 0, dilation= 1,
                                     return_indices= False)
                                     
    

    pooling_3: _layer = nn.MaxPool2d(kernel_size= (2, 2), stride= 1,
                                     padding= 0, dilation= 1,
                                     return_indices= False)
                                     

    pooling_4: _layer = nn.MaxPool2d(kernel_size= (2, 2), stride= 2, 
                                     padding= 0, dilation= 1,
                                     return_indices= False)
                                     

                                            
    linear_1: _layer = nn.Linear(in_features= 256 * 122 * 122, out_features= 64, bias= True)    
    linear_2: _layer = nn.Linear(in_features= 64, out_features= 32, bias= True)                  
    linear_3: _layer = nn.Linear(in_features= 32, out_features= 16, bias= True)                 
    linear_4: _layer = nn.Linear(in_features= 16, out_features= 8, bias= True)                  
    linear_5: _layer = nn.Linear(in_features= 8, out_features= 4, bias= True)                 

 
    #@: Activation functions
    relu: _activation = nn.ReLU()
    softmax: _activation = nn.Softmax()
    tanh: _activation = nn.Tanh()


    #@: Neuron's Characteristics
    flatten = nn.Flatten()                                                          
    dropout = nn.Dropout(p= 0.3) 

## _Custom CNN Model_

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, num_classes: int) -> None:
        super(CNNModel, self).__init__()
        
        self.convolution_layers = nn.Sequential(
            CNNHyperParams.convolutional_1,
            CNNHyperParams.relu,
            CNNHyperParams.pooling_1,

            CNNHyperParams.convolutional_2,
            CNNHyperParams.relu,
            CNNHyperParams.pooling_2,

            CNNHyperParams.convolutional_3,
            CNNHyperParams.relu,
            CNNHyperParams.pooling_3,

            CNNHyperParams.convolutional_4,
            CNNHyperParams.relu,
            CNNHyperParams.pooling_4
        )


        self.linear_layers = nn.Sequential(
            CNNHyperParams.linear_1,
            CNNHyperParams.relu,

            CNNHyperParams.linear_2,
            CNNHyperParams.relu,

            CNNHyperParams.linear_3,
            CNNHyperParams.relu,

            CNNHyperParams.linear_4,
            CNNHyperParams.relu,

            CNNHyperParams.linear_5
        )



    def forward(self, x: torch.Tensor) -> torch.Tensor:
        #print(x.shape)
        x: torch.Tensor = self.convolution_layers(x)
        #print(x.shape)
        #x: torch.Tensor = CNNHyperParams.dropout(x)
        #print(x.shape)
        x: torch.Tensor = CNNHyperParams.flatten(x)
        #print(x.shape)
        x: torch.Tensor = self.linear_layers(x)
        #print(x.shape)
        return x

## _Class Train_Validate_

In [ ]:
class Train_validate:
    def __init__(self, model: _model, train_loader: _loader, 
                                      test_loader: _loader, 
                                      criterion: _criterion,
                                      optimizer: _optimizor ) -> None:
        self.model = model
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.criterion = criterion
        self.optimizer = optimizer 
    
    

    def __repr__(self) -> str(dict[str, str]):
        return str({
            item: value for item, value in zip(['Module', 'Name', 'ObjectID'], [self.__module__, type(self).__name__, hex(id(self))])
        })


    __str__ = __repr__



    @classmethod
    def train_epoch(cls, model: _model, dataloader: _loader, 
                         optimizer: _optimizor, criterion: _criterion) -> tuple[float, float]:
        model.train()
        total_correct: int = 0
        total_loss: float = 0.0
        total_examples: int = 0
        
        for index, data in enumerate(dataloader):
            X, y = data
            X, y = X.to(DEVICE), y.to(DEVICE)
            y_hat = model(X.float())
            loss = criterion(y_hat, y.long())
            
            #@: back propogation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #@: end back propogation

            total_examples += y.size(0)
            total_loss += loss.item()
            total_correct += (torch.argmax(y_hat, 1) == y).sum().item()

        return total_loss / len(dataloader), total_correct / total_examples 

        


    @classmethod
    def test_epoch(cls, model: _model, dataloader: _loader, criterion: _criterion) -> tuple[float, float]:
        model.eval()
        total_correct: int = 0
        total_examples: int = 0
        total_loss: float = 0.0
        
        for index, data in enumerate(dataloader):
            X, y = data
            X, y = X.to(DEVICE), y.to(DEVICE)
            y_hat = model(X.float())
            loss = criterion(y_hat, y.long())

            total_examples += y.size(0)
            total_loss += loss.item()
            total_correct += (torch.argmax(y_hat, 1) == y).sum().item()

        return total_loss / len(dataloader), total_correct / total_examples




    def fit(self, epochs: int) -> _text:
        total_train_accuracy: list[float] = []
        total_test_accuracy: list[float] = []

        for epoch in range(epochs):
            print("------ Epoch {:02d} ------".format(epoch))
            
            #@: training loss + accs
            loss, train_acc = self.train_epoch(model= self.model, dataloader= tqdm(self.train_loader, desc= 'Training', unit= 'batch'), optimizer= self.optimizer, criterion= self.criterion)
            total_train_accuracy.append(train_acc)
            print("Train Loss: {:.04f}, Accuracy: {:.04f}".format(loss, train_acc))
            
            #@: testing loss + accs
            loss, test_acc = self.test_epoch(model= self.model, dataloader= self.test_loader, criterion= self.criterion)
            total_test_accuracy.append(test_acc) 
            print("Test  Loss: {:.04f}, Accuracy: {:.04f}".format(loss, test_acc))

        #@: total accuracies:
        print(f'Training Accuracy: {sum(total_train_accuracy) / len(total_train_accuracy)}')
        print(f'Testing Accuracy: {sum(total_test_accuracy) / len(total_test_accuracy)}')

# DRIVER CODE

In [ ]:
path: _path = '/content/drive/MyDrive/BRAIN_MRI/'
sub_path: list[str] = ['Testing', 'Training']

categories: list[str] = [
    'glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor'
]


training_data: object = BrainDataset(path= path,
                                     sub_path= sub_path[1],
                                     categories= categories,
                                     img_resolution= 64,
                                     transform= BrainPreprocess.train_transform_container())


testing_data: object = BrainDataset(path= path,
                                    sub_path= sub_path[0],
                                    categories= categories, 
                                    img_resolution= 64,
                                    transform= BrainPreprocess.test_transform_container())
    
    
training_data_loader: _loader = DataLoader(dataset= training_data, 
                                           batch_size= 8, 
                                           shuffle= True, 
                                           #num_workers= 4,
                                           pin_memory= True)


testing_data_loader: _loader = DataLoader(dataset= testing_data,
                                          batch_size= 8, 
                                          shuffle= False,
                                          #num_workers= 2,
                                          pin_memory= True)


data_loader_dict: dict[str, object|_loader] = {
    'training data'  : training_data,
    'training loader': training_data_loader,
    'testing data'   : testing_data,
    'testing loader' : testing_data_loader
}

print(data_loader_dict)

{'training data': {'Module': '__main__', 'Name': 'BrainDataset', 'ObjectID': '0x7f2bcf3b14d0'}, 'training loader': <torch.utils.data.dataloader.DataLoader object at 0x7f2bcec121d0>, 'testing data': {'Module': '__main__', 'Name': 'BrainDataset', 'ObjectID': '0x7f2bcec12050'}, 'testing loader': <torch.utils.data.dataloader.DataLoader object at 0x7f2bcec12350>}


In [ ]:
cnn_model = CNNModel(3)
print(cnn_model)

CNNModel(
  (convolution_layers): Sequential(
    (0): Conv2d(2, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=1, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=1, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(2, 2), stride=1, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=3810304, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=16, bias=True)
    (5): ReLU()
    (6): Linear(in_feature

In [ ]:
 def get_default_device() -> Any:
    if torch.cuda.is_available():
        return torch.device('cuda')
    return torch.device('cpu')

DEVICE: Any = get_default_device()

cnn_model.to(DEVICE)

training_data_loader: _loader =  GPU_Accerlaration(training_data_loader, DEVICE)
testing_data_loader: _loader = GPU_Accerlaration(testing_data_loader, DEVICE)

gpu_loader_dict: dict[str, _loader] = {
    'GPU Training DataLoader': training_data_loader, 
    'GPU Testing DataLoader': testing_data_loader,
    'GPU': torch.cuda.get_device_name(0) 
}

print(gpu_loader_dict)

{'GPU Training DataLoader': {'Module': '__main__', 'Name': 'GPU_Accerlaration', 'ObjectID': '0x7f2bcf3b1590'}, 'GPU Testing DataLoader': {'Module': '__main__', 'Name': 'GPU_Accerlaration', 'ObjectID': '0x7f2bcf3b1fd0'}, 'GPU': 'Tesla T4'}


In [ ]:
train_validate: object = Train_validate(model= cnn_model, 
                                        train_loader= training_data_loader, 
                                        test_loader= testing_data_loader, 
                                        criterion= CNNHyperParams.criterion, 
                                        optimizer= torch.optim.Adam(cnn_model.parameters(), lr= 0.1))

train_validate.fit(10)






Training:   0%|          | 0/50 [00:00<?, ?batch/s]

------ Epoch 00 ------







Training:   2%|▏         | 1/50 [00:00<00:25,  1.89batch/s]




Training:   4%|▍         | 2/50 [00:00<00:24,  1.99batch/s]




Training:   6%|▌         | 3/50 [00:01<00:23,  2.03batch/s]




Training:   8%|▊         | 4/50 [00:01<00:21,  2.09batch/s]




Training:  10%|█         | 5/50 [00:02<00:21,  2.10batch/s]




Training:  12%|█▏        | 6/50 [00:02<00:20,  2.12batch/s]




Training:  14%|█▍        | 7/50 [00:03<00:20,  2.13batch/s]




Training:  16%|█▌        | 8/50 [00:03<00:19,  2.15batch/s]




Training:  18%|█▊        | 9/50 [00:04<00:18,  2.19batch/s]




Training:  20%|██        | 10/50 [00:04<00:18,  2.17batch/s]




Training:  22%|██▏       | 11/50 [00:05<00:18,  2.16batch/s]




Training:  24%|██▍       | 12/50 [00:05<00:17,  2.16batch/s]




Training:  26%|██▌       | 13/50 [00:06<00:16,  2.19batch/s]




Training:  28%|██▊       | 14/50 [00:06<00:16,  2.19batch/s]




Training:  30%|███       | 15/50 [00:06<00:15,  2.23batch/s]




Training:  32%|███▏      | 16

Train Loss: nan, Accuracy: 0.2538







Training:   0%|          | 0/50 [00:00<?, ?batch/s]

...

Train Loss: nan, Accuracy: 0.2538







Training:   0%|          | 0/50 [00:00<?, ?batch/s]

Test  Loss: nan, Accuracy: 0.2538
------ Epoch 03 ------







Training:   2%|▏         | 1/50 [00:00<00:21,  2.30batch/s]




Training:   4%|▍         | 2/50 [00:00<00:21,  2.26batch/s]




Training:   6%|▌         | 3/50 [00:01<00:21,  2.21batch/s]




Training:   8%|▊         | 4/50 [00:01<00:20,  2.20batch/s]




Training:  10%|█         | 5/50 [00:02<00:20,  2.20batch/s]




Training:  12%|█▏        | 6/50 [00:02<00:19,  2.22batch/s]




Training:  14%|█▍        | 7/50 [00:03<00:19,  2.22batch/s]




Training:  16%|█▌        | 8/50 [00:03<00:18,  2.22batch/s]




Training:  18%|█▊        | 9/50 [00:04<00:18,  2.20batch/s]




Training:  20%|██        | 10/50 [00:04<00:18,  2.19batch/s]




Training:  22%|██▏       | 11/50 [00:05<00:17,  2.19batch/s]




Training:  24%|██▍       | 12/50 [00:05<00:18,  2.11batch/s]




Training:  26%|██▌       | 13/50 [00:05<00:17,  2.11batch/s]




Training:  28%|██▊       | 14/50 [00:06<00:16,  2.15batch/s]




Training:  30%|███       | 15/50 [00:06<00:16,  2.12batch/s]




Training:  32%|███▏      | 16

KeyboardInterrupt: ignored